In [ ]:
import numpy as np

class IMU:
    def __init__(self):
        self.time = 0
        self.dtheta = np.zeros(3)  # Angular velocity increments
        self.dvel = np.zeros(3)    # Velocity increments
        self.dt = 0                # Time difference
        self.odovel = 0            # Optional velocity data

class ImuFileLoader:
    def __init__(self, filename, columns, rate=200):
        self.file_loader = FileLoader(filename, columns, 'TEXT')
        self.columns = columns
        self.dt = 1.0 / rate
        self.imu = IMU()
        self.imu_pre = IMU()
    
    def next(self):
        data = self.file_loader.load()
        if not data:
            return None
        
        self.imu_pre = self.imu
        self.imu = IMU()
        self.imu.time = data[0]
        self.imu.dtheta = np.array(data[1:4])
        self.imu.dvel = np.array(data[4:7])
        
        dt = self.imu.time - self.imu_pre.time
        if dt < 0.1:
            self.imu.dt = dt
        else:
            self.imu.dt = self.dt
        
        # Handle optional velocity data
        if len(data) > 7:
            self.imu.odovel = data[7] * self.imu.dt
        
        return self.imu

    def starttime(self):
        initial_pos = self.file_loader.file.tell()
        self.file_loader.file.seek(0)
        start_time = float(self.file_loader.file.readline().split(',')[0])
        self.file_loader.file.seek(initial_pos)
        return start_time

    def endtime(self):
        initial_pos = self.file_loader.file.tell()
        self.file_loader.file.seek(0, 2)  # Move to end of file
        self.file_loader.file.seek(self.file_loader.file.tell() - 100, 0)  # Go back 100 bytes to ensure in data
        lines = self.file_loader.file.readlines()
        last_line = lines[-1].strip()
        end_time = float(last_line.split(',')[0])
        self.file_loader.file.seek(initial_pos)
        return end_time

# Example usage:
loader = ImuFileLoader('imu_data.txt', 8)
print(loader.starttime())
imu_data = loader.next()
print(imu_data.time, imu_data.dtheta, imu_data.dvel, imu_data.dt, imu_data.odovel)
